# 06. MÔ HÌNH HỒI QUY PANEL & KIỂM ĐỊNH ECONOMETRICS

## Unit Root Test + Hausman Test + Panel Regression (FE/RE)

**Bao gồm 3 bước:**

### 1. Unit Root Test (ADF - Augmented Dickey-Fuller)
- Kiểm tra tính dừng (stationarity) của chuỗi thời gian FDI, GDP_Growth, Trade_Openness
- H0: Chuỗi có unit root (non-stationary)
- H1: Chuỗi không có unit root (stationary)
- Nếu p-value < 0.05 → Chuỗi dừng, có thể dùng trực tiếp trong mô hình

### 2. Hausman Test
- So sánh Fixed Effects (FE) và Random Effects (RE) model
- H0: RE model phù hợp (không có correlation giữa individual effects và regressors)
- H1: FE model phù hợp (có correlation)
- Nếu p-value < 0.05 → Chọn FE, ngược lại chọn RE

### 3. Panel Regression
- **Fixed Effects:** $Y_{it} = \alpha_i + \beta_1 FDI_{it} + \beta_2 Trade_{it} + \epsilon_{it}$
- **Random Effects:** $Y_{it} = \alpha + \beta_1 FDI_{it} + \beta_2 Trade_{it} + u_i + \epsilon_{it}$

**Dữ liệu:** Panel data 9 quốc gia × 25 năm = 225 observations

---

In [1]:
# Install required package
%pip install linearmodels -q
print("✅ linearmodels installed!")

Note: you may need to restart the kernel to use updated packages.
✅ linearmodels installed!


In [2]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
from linearmodels.panel import PanelOLS, RandomEffects
from scipy.stats import chi2
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported!")

✅ Libraries imported!


In [3]:
df = pd.read_csv('../01_Data/02_Processed/ASEAN_FDI_GDP_Data_Final.csv')
df = df.set_index(['Country', 'Year'])
print(f"✅ Data loaded & indexed: {df.shape}")

✅ Data loaded & indexed: (225, 6)


## 1. Unit Root Test (ADF)

In [4]:
def adf_test(series, var_name):
    result = adfuller(series.dropna(), autolag='AIC')
    print(f"\n📊 ADF Test for {var_name}:")
    print(f"  ADF Statistic: {result[0]:.4f}")
    print(f"  p-value: {result[1]:.4f}")
    print(f"  Critical Values: {result[4]}")
    if result[1] < 0.05:
        print(f"  ✅ {var_name} is STATIONARY (p < 0.05)")
    else:
        print(f"  ⚠️ {var_name} is NON-STATIONARY (p >= 0.05) → Consider first differencing")
    return result[1] < 0.05

# Test each variable
fdi_stationary = adf_test(df['FDI'], 'FDI')
gdp_stationary = adf_test(df['GDP_Growth'], 'GDP_Growth')
trade_stationary = adf_test(df['Trade_Openness'], 'Trade_Openness')


📊 ADF Test for FDI:
  ADF Statistic: -3.2769
  p-value: 0.0159
  Critical Values: {'1%': np.float64(-3.461429147102264), '5%': np.float64(-2.875206950587861), '10%': np.float64(-2.5740544497344002)}
  ✅ FDI is STATIONARY (p < 0.05)

📊 ADF Test for GDP_Growth:
  ADF Statistic: -4.2993
  p-value: 0.0004
  Critical Values: {'1%': np.float64(-3.461878735881654), '5%': np.float64(-2.875403665910809), '10%': np.float64(-2.574159410430839)}
  ✅ GDP_Growth is STATIONARY (p < 0.05)

📊 ADF Test for Trade_Openness:
  ADF Statistic: -2.4800
  p-value: 0.1204
  Critical Values: {'1%': np.float64(-3.459884913337196), '5%': np.float64(-2.8745310704320794), '10%': np.float64(-2.573693840082908)}
  ⚠️ Trade_Openness is NON-STATIONARY (p >= 0.05) → Consider first differencing


## 2. Hausman Test

In [5]:
# Fixed Effects Model
fe_model = PanelOLS(df['GDP_Growth'], df[['FDI', 'Trade_Openness']], entity_effects=True)
fe_results = fe_model.fit()

# Random Effects Model
re_model = RandomEffects(df['GDP_Growth'], df[['FDI', 'Trade_Openness']])
re_results = re_model.fit()

# Hausman Test
coef_diff = fe_results.params - re_results.params
cov_diff = fe_results.cov - re_results.cov
hausman_stat = coef_diff.T @ np.linalg.pinv(cov_diff) @ coef_diff
p_value = 1 - chi2.cdf(hausman_stat, df=len(coef_diff))

print("\n📊 Hausman Test:")
print(f"  Hausman Statistic: {hausman_stat:.4f}")
print(f"  p-value: {p_value:.4f}")
if p_value < 0.05:
    print("  ✅ Use FIXED EFFECTS Model (p < 0.05)")
    preferred_model = 'FE'
else:
    print("  ✅ Use RANDOM EFFECTS Model (p >= 0.05)")
    preferred_model = 'RE'


📊 Hausman Test:
  Hausman Statistic: 0.3892
  p-value: 0.8232
  ✅ Use RANDOM EFFECTS Model (p >= 0.05)


## 3. Panel Regression Results

In [6]:
print("\n" + "="*60)
print("FIXED EFFECTS MODEL")
print("="*60)
print(fe_results)

print("\n" + "="*60)
print("RANDOM EFFECTS MODEL")
print("="*60)
print(re_results)


FIXED EFFECTS MODEL
                          PanelOLS Estimation Summary                           
Dep. Variable:             GDP_Growth   R-squared:                        0.0464
Estimator:                   PanelOLS   R-squared (Between):              0.5534
No. Observations:                 225   R-squared (Within):               0.0464
Date:                Wed, Dec 10 2025   R-squared (Overall):              0.4406
Time:                        15:03:45   Log-likelihood                   -540.54
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      5.2097
Entities:                           9   P-value                           0.0062
Avg Obs:                       25.000   Distribution:                   F(2,214)
Min Obs:                       25.000                                           
Max Obs:                       25.000   F-statistic (robust):             5.2097
       

## 4. Export Results

In [7]:
# Save to text file
with open('../03_Results/Panel_Regression_Results.txt', 'w') as f:
    f.write("UNIT ROOT TESTS (ADF)\n")
    f.write("="*60 + "\n")
    f.write(f"FDI: {'Stationary' if fdi_stationary else 'Non-stationary'}\n")
    f.write(f"GDP_Growth: {'Stationary' if gdp_stationary else 'Non-stationary'}\n")
    f.write(f"Trade_Openness: {'Stationary' if trade_stationary else 'Non-stationary'}\n\n")
    
    f.write("HAUSMAN TEST\n")
    f.write("="*60 + "\n")
    f.write(f"Hausman Statistic: {hausman_stat:.4f}\n")
    f.write(f"p-value: {p_value:.4f}\n")
    f.write(f"Preferred Model: {preferred_model}\n\n")
    
    f.write("FIXED EFFECTS MODEL\n")
    f.write("="*60 + "\n")
    f.write(str(fe_results) + "\n\n")
    
    f.write("RANDOM EFFECTS MODEL\n")
    f.write("="*60 + "\n")
    f.write(str(re_results) + "\n")

print("\n✅ Panel regression results saved to text file!")


✅ Panel regression results saved to text file!


In [8]:
# Save coefficients to CSV
panel_results = pd.DataFrame({
    'Variable': fe_results.params.index,
    'FE_Coef': fe_results.params.values,
    'FE_pvalue': fe_results.pvalues.values,
    'RE_Coef': re_results.params.values,
    'RE_pvalue': re_results.pvalues.values
})
panel_results.to_csv('../03_Results/Panel_Coefficients.csv', index=False)
print("✅ Coefficients saved to CSV!")

✅ Coefficients saved to CSV!
